In [2]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Mouse C3KO-RichLi snRNA
####  2023-05-16 by Yi Zhao (Texas Heart Institute, US)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Initiate  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
Ver <- '0'
Step <- 'PART03_Filter_Cells'
Project <- '2023_neoc3ko_rli'

Code_dir <- paste0('/Volumes/shire/project/', Project, '/code/mouse_v', Ver, '/')

source(Sys.readlink(paste0(Code_dir, 'src/bioinformatics.R')))
source(Sys.readlink(paste0(Code_dir, 'src/scRNAseq.R')))
source(Sys.readlink(paste0(Code_dir, 'src/scATACseq.R')))
source(paste0(Code_dir, 'mouse_v', Ver, '.helper_functions.R'))

InitiateProject('Rivendell', Ver, Step, 'mouse', Project, 'shire')
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [3]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Load data  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
merged.flt.srt <- readRDS('integrated/PART02.merged.srt.rds')
studies <- levels(merged.flt.srt$study)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Set hard quality filters  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
## Filter cells with less than 200 umi(counts)/cell
cutoff1 <- 200
pct_filtered <- rep(NA, L(studies))
names(pct_filtered) <- studies
p1 <- VlnPlot(merged.flt.srt,
              features = c('nCount_RNA'), log = T,
              group.by = 'sample', pt.size = -1, ncol = 1) +
        geom_hline(yintercept = cutoff1) +
        NoLegend()
for(i in 1:L(studies)){
        n_filtered <- sum(merged.flt.srt$nCount_RNA < cutoff1 & merged.flt.srt$study == studies[i])
        pct_filtered[i] <- n_filtered*100/sum(merged.flt.srt$study == studies[i])
}
## Filter cells with less than 150 genes/cell
cutoff2 <- 150
pct_filtered <- rep(NA, L(studies))
names(pct_filtered) <- studies
p2 <- VlnPlot(merged.flt.srt,
              features = c('nFeature_RNA'), log = T,
              group.by = 'sample', pt.size = -1, ncol = 1) +
        geom_hline(yintercept = cutoff2) +
        NoLegend()
for(i in 1:L(studies)){
        n_filtered <- sum(merged.flt.srt$nFeature_RNA < cutoff2 & merged.flt.srt$study == studies[i])
        pct_filtered[i] <- n_filtered*100/sum(merged.flt.srt$study == studies[i])
}
## Filter cells with higher than 5% mitochondrial counts/cell
cutoff3 <- 12
pct_filtered <- rep(NA, L(studies))
names(pct_filtered) <- studies
p3 <- VlnPlot(merged.flt.srt,
              features = c('pct_mito_RNA'),
              group.by = 'sample', pt.size = -1, ncol = 1) +
        geom_hline(yintercept = cutoff3) +
        NoLegend()
for(i in 1:L(studies)){
        n_filtered <- sum(merged.flt.srt$pct_mito_RNA > cutoff3 & merged.flt.srt$study == studies[i])
        pct_filtered[i] <- n_filtered*100/sum(merged.flt.srt$study == studies[i])
}

cells_toss_1 <- Cells(merged.flt.srt)[
        merged.flt.srt$nCount_RNA < cutoff1 |
                merged.flt.srt$nFeature_RNA < cutoff2 |
                merged.flt.srt$pct_mito_RNA > cutoff3
]
L(cells_toss_1) ## 2454
L(cells_toss_1)*100/ncol(merged.flt.srt) ## 5.440638510143%

p4 <- DimPlot(merged.flt.srt, cols = 'grey75', reduction = 'hmn_umap', raster = T,
              cells.highlight = cells_toss_1, cols.highlight = 'red', sizes.highlight = 0.001, pt.size = 0.001) +
        labs(title = paste0('Total cells: ', ncol(merged.flt.srt), '  Cells filtered (red): ', L(cells_toss_1))) +
        NoLegend() +
        theme(aspect.ratio = 1, axis.text = element_blank(), axis.ticks = element_blank())

PlotPDF('01.1.merged_consensus.hard_cutoff.vln', 12, 6)
p1 | p2 | p3
dev.off()
PlotPDF('01.2.merged_consensus.hard_cutoff.dim', 10, 10)
p4
dev.off()
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



[1] 7627

[1] 6.568432

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



pdf 
  2

pdf 
  2

In [4]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Set dynamic quality filters  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
L(cells_toss_1) ## 1880

## Remove bad cells that did not pass hard filters before setting dynamic filters
merged.flt.srt2 <- merged.flt.srt[, !Cells(merged.flt.srt) %in% cells_toss_1]
all_samples <- U(merged.flt.srt2$sample)
L(all_samples) ## 7

## Set filter for max nCount
cutoff_upper <- c()
cutoff_lower <- c()
cells_toss_2 <- c()
for(i in 1:L(all_samples)){
        print(i)
        cells_in_group <- Cells(merged.flt.srt2)[merged.flt.srt2$sample == all_samples[i]]
        GetOutlier(merged.flt.srt2$nCount_RNA[cells_in_group])
        cutoff_upper[i] <- GetOutlier(merged.flt.srt2$nCount_RNA[cells_in_group])[2]
        cutoff_lower[i] <- GetOutlier(merged.flt.srt2$nCount_RNA[cells_in_group])[1]
        if(cutoff_lower[i] < cutoff1){cutoff_lower[i] <- cutoff1}
        cells_toss_2 <- c(cells_toss_2,
                          cells_in_group[merged.flt.srt2$nCount_RNA[cells_in_group] > cutoff_upper[i] |
                                                 merged.flt.srt2$nCount_RNA[cells_in_group] < cutoff_lower[i]])
}
L(cells_toss_2) ## 3434
p <- list()
for(i in 1:L(all_samples)){
        p[[i]] <- ggplot(merged.flt.srt2@meta.data[merged.flt.srt2$sample == all_samples[i], ],
                         aes(x = sample, y = nCount_RNA)) +
                geom_violin(fill = mycol_30[3]) +
                geom_hline(yintercept = c(cutoff_upper[i], cutoff_lower[i]), color = mycol_14[1], size = 1) +
                labs(title = paste0('Sample # ', all_samples[i]),
                     subtitle = paste0(round(cutoff_lower[i], 0) , ' < UMI < ', round(cutoff_upper[i], 0))) +
                scale_y_continuous(limits = c(0, 5e4))
}
p2 <- wrap_plots(p[order(cutoff_upper)], ncol = 5) &
        NoLegend() &
        WhiteBackground() &
        theme(aspect.ratio = 2,
              axis.title = element_blank(),
              axis.text = element_blank(),
              axis.ticks = element_blank(),
              axis.line = element_line())
p3 <- DimPlot(merged.flt.srt, cols = 'grey75', reduction = 'hmn_umap', raster = T,
              pt.size = 0.001, sizes.highlight = 0.001,
              cells.highlight = cells_toss_2, cols.highlight = 'red') +
        labs(title = paste0('Total cells: ', ncol(merged.flt.srt), '  Cells filtered (red): ', L(cells_toss_2))) +
        NoLegend() +
        theme(aspect.ratio = 1, axis.text = element_blank(), axis.ticks = element_blank())
PlotPDF('02.1.merged_consensus.dynamic_cutoff_nCount.vln', 8, 16)
p2
dev.off()
PlotPDF('02.2.merged_consensus.dynamic_cutoff_nCount.dim', 10, 10)
p3
dev.off()

## Set filter for max nFeature
cutoff_upper <- c()
cutoff_lower <- c()
cells_toss_3 <- c()
for(i in 1:L(all_samples)){
        print(i)
        cells_in_group <- Cells(merged.flt.srt2)[merged.flt.srt2$sample == all_samples[i]]
        GetOutlier(merged.flt.srt2$nCount_RNA[cells_in_group])
        cutoff_upper[i] <- GetOutlier(merged.flt.srt2$nFeature_RNA[cells_in_group])[2]
        cutoff_lower[i] <- GetOutlier(merged.flt.srt2$nFeature_RNA[cells_in_group])[1]
        if(cutoff_lower[i] < cutoff2){cutoff_lower[i] <- cutoff2}
        cells_toss_3 <- c(cells_toss_3,
                          cells_in_group[merged.flt.srt2$nFeature_RNA[cells_in_group] > cutoff_upper[i] |
                                                 merged.flt.srt2$nFeature_RNA[cells_in_group] < cutoff_lower[i]])
}
L(cells_toss_3) ## 1812
p <- list()
for(i in 1:L(all_samples)){
        p[[i]] <- ggplot(merged.flt.srt2@meta.data[merged.flt.srt2$sample == all_samples[i], ],
                         aes(x = sample, y = nFeature_RNA)) +
                geom_violin(fill = mycol_30[3]) +
                geom_hline(yintercept = c(cutoff_upper[i], cutoff_lower[i]), color = mycol_14[1], size = 1) +
                labs(title = paste0('Sample # ', all_samples[i]),
                     subtitle = paste0(round(cutoff_lower[i], 0) , ' < Gene < ', round(cutoff_upper[i], 0))) +
                scale_y_continuous(limits = c(0, 1.5e4))
}
p2 <- wrap_plots(p[order(cutoff_upper)], ncol = 5) &
        NoLegend() &
        WhiteBackground() &
        theme(aspect.ratio = 2,
              axis.title = element_blank(),
              axis.text = element_blank(),
              axis.ticks = element_blank(),
              axis.line = element_line())
p3 <- DimPlot(merged.flt.srt, cols = 'grey75', reduction = 'hmn_umap', raster = T,
              pt.size = 0.001, sizes.highlight = 0.001,
              cells.highlight = cells_toss_3, cols.highlight = 'red') +
        labs(title = paste0('Total cells: ', ncol(merged.flt.srt), '  Cells filtered (red): ', L(cells_toss_3))) +
        NoLegend() +
        theme(aspect.ratio = 1, axis.text = element_blank(), axis.ticks = element_blank())
PlotPDF('03.1.merged_consensus.dynamic_cutoff_nFeature.vln', 10, 16)
p2
dev.off()
PlotPDF('03.2.merged_consensus.dynamic_cutoff_nFeature.dim', 10, 10)
p3
dev.off()
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

[1] 7627

[1] 8

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8


[1] 6359

Warning message:
"Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead."
Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Warning message:
"Removed 7 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 8 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 5 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 4 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 17 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 11 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 36 rows containing non-finite values (`stat_ydensity()`)."
Warning message:
"Removed 11 rows containing non-finite values (`stat_ydensity()`)."


...

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8


[1] 4906

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



pdf 
  2

pdf 
  2

In [5]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Filter cells  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
cells_toss <- U(c(cells_toss_1, cells_toss_2, cells_toss_3))
L(cells_toss) ## 5330
p1 <- DimPlot(merged.flt.srt, cols = 'grey75', reduction = 'hmn_umap', raster = T, pt.size = 0.001,
              cells.highlight = cells_toss, cols.highlight = 'red', sizes.highlight = 0.001) +
        labs(title = paste0('Total cells: ', ncol(merged.flt.srt),
                            '  Cells filtered (red): ', L(cells_toss),
                            ' (', round(L(cells_toss)*100/ncol(merged.flt.srt)),'%)')) +
        NoLegend() +
        theme(aspect.ratio = 1, axis.text = element_blank(), axis.ticks = element_blank())
PlotPDF('05.1.merged_consensus.all_cutoff', 10, 10)
p1
dev.off()

merged.flt.srt$LowQual <- F
merged.flt.srt$LowQual[cells_toss_3] <- 'cells_toss_3'
merged.flt.srt$LowQual[cells_toss_2] <- 'cells_toss_2'
merged.flt.srt$LowQual[cells_toss_1] <- 'cells_toss_1'

p1 <- CountCellBarPlot(merged.flt.srt, group.var = 'study', stack.var = 'LowQual', stack.color = mycol_10)$plot
p2 <- CountCellBarPlot(merged.flt.srt, group.var = 'study', stack.var = 'LowQual',
                       stack.color = mycol_10, percentage = T)$plot
PlotPDF('05.2.merged_consensus.pct_toss_study', 10, 15, onefile = T)
wrap_plots(p1, p2, ncol = 1)
dev.off()

p1 <- CountCellBarPlot(merged.flt.srt, group.var = 'sample', stack.var = 'LowQual', stack.color = mycol_10)$plot
p2 <- CountCellBarPlot(merged.flt.srt, group.var = 'sample', stack.var = 'LowQual',
                       stack.color = mycol_10, percentage = T)$plot
PlotPDF('05.3.merged_consensus.pct_toss_group', 20, 15, onefile = T)
wrap_plots(p1, p2, ncol = 1)
dev.off()

cell_toss.list <- list(cells_toss_1, cells_toss_2, cells_toss_3)

rm(merged.flt.srt2, p, p1, p2, p3, p4)
gc()
merged.flt.srt <- merged.flt.srt[, !Cells(merged.flt.srt) %in% cells_toss]
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
saveRDS(cell_toss.list, 'analysis/PART03.cell_filtered.low_quality_cells.rds')
saveRDS(merged.flt.srt@meta.data, 'integrated/PART03.merged.flt.srt_meta.rds')
saveRDS(merged.flt.srt, 'integrated/PART03.merged.flt.srt.rds')
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

[1] 14481

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



pdf 
  2

Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(stack.var)

  # Now:
  data %>% select(all_of(stack.var))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."
Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(group.var)

  # Now:
  data %>% select(all_of(group.var))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."
`summarise()` has grouped output by 'LowQual'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'LowQual'. You can override using the
`.groups` argument.


pdf 
  2

`summarise()` has grouped output by 'LowQual'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'LowQual'. You can override using the
`.groups` argument.


pdf 
  2

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,15129927,808.1,27528598,1470.2,NA,27528598,1470.2
Vcells,760656464,5803.4,2098483445,16010.2,131072,1748626871,13341.0
